In [ ]:
from pathlib import Path

import bioread
import matplotlib.pyplot as plt
import neurokit2 as nk
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Read the .acq file
data = bioread.read_file('../.data/AdVitam/Exp4/Raw/Physio/BioPac/01_AC16.acq')

In [ ]:
data.event_markers

[EventMarker Segment 1: idx: 0 channel: None date_created_utc: Unknown type_code: None,
 EventMarker Baseline Start: idx: 140985 channel: None date_created_utc: Unknown type_code: None,
 EventMarker Baseline End: idx: 468730 channel: None date_created_utc: Unknown type_code: None,
 EventMarker Training Start: idx: 669560 channel: None date_created_utc: Unknown type_code: None,
 EventMarker Training End: idx: 888195 channel: None date_created_utc: Unknown type_code: None,
 EventMarker Block1 Start: idx: 1080925 channel: None date_created_utc: Unknown type_code: None,
 EventMarker Block1 End: idx: 2803700 channel: None date_created_utc: Unknown type_code: None,
 EventMarker Block2 Start: idx: 3192855 channel: None date_created_utc: Unknown type_code: None,
 EventMarker Block2 End: idx: 4683925 channel: None date_created_utc: Unknown type_code: None]

In [ ]:
for marker in data.event_markers:
    print(marker.time_index / 60)

0.0
2.3497500000000002
7.812166666666667
11.159333333333333
14.80325
18.015416666666667
46.72833333333333
53.21425
78.06541666666666


In [ ]:
data.channels

[Channel EDA (0 - 35 Hz): 4726500 samples, 1000.0 samples/sec, loaded: True,
 Channel ECG (.5 - 35 Hz): 4726500 samples, 1000.0 samples/sec, loaded: True,
 Channel Respiration: 4726500 samples, 1000.0 samples/sec, loaded: True]

**BioPac .acq files** contain synchronized physiological data recorded using BioPac hardware and AcqKnowledge software. These files store multiple channels of continuous physiological measurements with precise timing and metadata.

**Basic Components:**
- **Channels** - Individual physiological signals (ECG, EDA, RESP)
- **Sampling rates** - Data collection frequency (typically 1000 Hz)
- **Event markers** - Timestamps marking experimental phases
- **Metadata** - Units, filters, calibration information

**Key Properties:**
- **Synchronized recording** - All channels sampled simultaneously
- **High temporal resolution** - 1000 samples per second = 1ms precision
- **Pre-filtered signals** - Hardware filters applied during acquisition
- **Scaled data** - Raw ADC values converted to physiological units

**Physiological Signals Explained**

| Signal | Full Name | What It Measures | Typical Units | Filter Range |
|--------|-----------|------------------|---------------|--------------|
| **ECG** | Electrocardiogram | Heart electrical activity | mV or μV | 0.5-35 Hz |
| **EDA** | Electrodermal Activity | Skin conductance/sweat response | μS or μmho | 0-35 Hz |
| **RESP** | Respiration | Breathing effort/chest movement | V or arbitrary | Variable |

**ECG (Electrocardiogram):**
- **Measures:** Electrical activity of the heart muscle
- **Placement:** Electrodes on chest/limbs detect heart depolarization
- **Information:** Heart rate, rhythm, stress indicators
- **Filter rationale:** 0.5 Hz removes baseline drift, 35 Hz removes muscle noise
- **Key features:** R-peaks for heart rate variability (HRV) analysis

**EDA (Electrodermal Activity):**
- **Measures:** Skin conductance changes due to sweat gland activity
- **Placement:** Electrodes on fingertips or palm
- **Information:** Sympathetic nervous system arousal, stress, cognitive load
- **Filter rationale:** 0-35 Hz captures slow autonomic responses, removes electrical noise
- **Components:** Tonic (baseline) and phasic (event-related) responses

**Respiration:**
- **Measures:** Breathing rate and depth
- **Method:** Strain gauge around chest/abdomen detects expansion
- **Information:** Breathing patterns, stress, relaxation state
- **Analysis:** Respiratory rate, variability, respiratory sinus arrhythmia (RSA)

**Duration and Timing Analysis**

| Phase | Expected Duration | Cumulative |
|-------|------------------|------------|
| Baseline | 5 minutes | 5 min |
| Training | 5 minutes | 10 min |
| Scenario 1 | 30 minutes | 40 min |
| Scenario 2 | 30 minutes | 70 min |
| **Total Core** | **70 minutes** | **70 min** |
| **Actual Recording** | **78.8 minutes** | **78.8 min** |

**Extra 8.8 minutes likely includes:**
- Pre-experiment setup and sensor stabilization
- Between-phase questionnaire completion
- Post-experiment data collection

**Essential bioread Information**

**What bioread provides:**
- **Direct .acq file reading** - No conversion needed
- **Channel access** - Individual signal data as NumPy arrays
- **Metadata extraction** - Units, sampling rates, filters
- **Event marker parsing** - Experimental phase timestamps

**File structure access:**
- **data.channels[i]** - Access individual channels (ECG, EDA, RESP)
- **channel.data** - Raw signal values as NumPy array
- **channel.samples_per_second** - Sampling frequency (Hz)
- **channel.units** - Physical measurement units
- **data.event_markers** - Experimental phase timestamps

In [ ]:
data.channels[2].data

array([8.78601074, 8.78540039, 8.78479004, ..., 1.01989746, 1.02203369,
       1.0244751 ], shape=(4726500,))

In [ ]:
# Quick data preview
print("=== DATA PREVIEW ===")
print("First 10 samples of each channel:")
for i, channel in enumerate(data.channels):
    print(f"{channel.name}: {channel.data[:10]}")

=== DATA PREVIEW ===
First 10 samples of each channel:
EDA (0 - 35 Hz): [1.43127441 1.43127441 1.43127441 1.42974854 1.42974854 1.42974854
 1.42974854 1.42822266 1.42822266 1.42822266]
ECG (.5 - 35 Hz): [-6.45111084 -6.40167236 -6.35375977 -6.30889893 -6.2677002  -6.23138428
 -6.2008667  -6.17645264 -6.15875244 -6.14776611]
Respiration: [8.78601074 8.78540039 8.78479004 8.78417969 8.78326416 8.78265381
 8.78204346 8.78143311 8.78051758 8.77990723]


# HRV Segmentation and Event Markers Analysis



In [ ]:
import bioread
import neurokit2 as nk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

# %% [markdown]
# ## 1. Event Marker Analysis Across All Files
#
# First, let's examine the event markers across all BioPac files to understand the experimental timing.

# %%


def extract_event_markers_from_file(file_path):
    """
    Extract event markers from a single BioPac file

    Returns:
        dict: File info and event markers
    """
    data = bioread.read_file(file_path)

    file_info = {
        'file_name': Path(file_path).name,
        'file_path': str(file_path),
        'total_duration_min': 0,
        'sampling_rate': 0,
        'n_channels': len(data.channels),
        'channels': []
    }

    # Get basic file info
    if data.channels:
        file_info['sampling_rate'] = data.channels[0].samples_per_second
        file_info['total_duration_min'] = len(
            data.channels[0].data) / (data.channels[0].samples_per_second * 60)
        file_info['channels'] = [ch.name for ch in data.channels]

    # Extract event markers with actual names
    markers = {}
    for marker in data.event_markers:
        marker_time_sec = marker.sample_index / file_info['sampling_rate']
        marker_time_min = marker_time_sec / 60

        # Clean marker text and use as key
        marker_text = marker.text.strip()
        time_key = f"{marker_text}_time_sec"
        time_min_key = f"{marker_text}_time_min"

        markers[time_key] = marker_time_sec
        markers[time_min_key] = marker_time_min

    # Combine file info and markers
    result = {**file_info, **markers}
    return result


def analyze_all_event_markers(data_directory):
    """
    Analyze event markers across all BioPac files in directory
    """
    data_path = Path(data_directory)
    acq_files = list(data_path.glob("*.acq"))

    print(f"Found {len(acq_files)} .acq files in {data_directory}")

    all_file_data = []

    for file_path in sorted(acq_files):
        print(f"Processing: {file_path.name}")
        file_data = extract_event_markers_from_file(file_path)
        all_file_data.append(file_data)

    # Convert to DataFrame
    df = pd.DataFrame(all_file_data)

    return df


# %%
# Analyze all files - UPDATE THIS PATH to your data directory
data_directory = "../.data/AdVitam/Exp4/Raw/Physio/BioPac/"

# Get event marker data for all files
event_markers_df = analyze_all_event_markers(data_directory)

# Display basic info
print("=== FILE OVERVIEW ===")
print(f"Total files processed: {len(event_markers_df)}")

# Show first few files
print("\n=== FIRST 5 FILES ===")
display_cols = ['file_name', 'total_duration_min',
                'sampling_rate', 'n_channels']
if len(event_markers_df) > 0:
    print(event_markers_df[display_cols].head())

# %% [markdown]
# ## 2. Event Marker Pattern Analysis

# %%


def analyze_event_patterns(df):
    """
    Analyze patterns in event markers to identify experimental phases
    """
    print("=== EVENT MARKER ANALYSIS ===")

    # Find all unique marker texts
    marker_text_cols = [col for col in df.columns if col.endswith('_text')]
    all_marker_texts = set()

    for col in marker_text_cols:
        unique_texts = df[col].dropna().unique()
        all_marker_texts.update(unique_texts)

    print(f"\nUnique marker texts found across all files:")
    for text in sorted(all_marker_texts):
        print(f"  - '{text}'")

    # Show marker timing patterns for first few files
    print(f"\n=== TIMING PATTERNS (First 3 files) ===")

    for idx in range(min(3, len(df))):
        row = df.iloc[idx]
        print(f"\nFile: {row['file_name']}")
        print(f"Total duration: {row['total_duration_min']:.1f} minutes")

        # Show all markers for this file
        marker_cols = [(col, col.replace('_text', '_time_min'))
                       for col in df.columns
                       if col.endswith('_text') and pd.notna(row[col])]

        for text_col, time_col in marker_cols:
            if time_col in df.columns:
                print(f"  {row[text_col]}: {row[time_col]:.1f} min")


# Analyze patterns
analyze_event_patterns(event_markers_df)

# %% [markdown]
# ## 3. Calculate Driving Phase Durations
#
# Based on the AdVitam protocol:
# - Baseline: 5 minutes
# - Training: 5 minutes
# - Block1: ~30 minutes (first driving scenario)
# - Block2: ~30 minutes (second driving scenario)

# %%


def calculate_driving_durations(df):
    """
    Calculate total driving duration (Block2 End - Block1 Start)
    """
    results = df.copy()
    results['total_driving_duration_min'] = np.nan
    results['block1_start_min'] = np.nan
    results['block2_end_min'] = np.nan

    for idx, row in results.iterrows():
        # Find Block1 Start and Block2 End times
        block1_start_time = None
        block2_end_time = None

        # Look for the specific time columns
        if 'Block1 Start_time_min' in row:
            block1_start_time = row['Block1 Start_time_min']
        if 'Block2 End_time_min' in row:
            block2_end_time = row['Block2 End_time_min']
        elif 'Block2 End _time_min' in row:  # Handle the space variant
            block2_end_time = row['Block2 End _time_min']

        # Store start and end times
        results.at[idx, 'block1_start_min'] = block1_start_time
        results.at[idx, 'block2_end_min'] = block2_end_time

        # Calculate total driving duration
        if block1_start_time is not None and block2_end_time is not None:
            total_duration = block2_end_time - block1_start_time
            results.at[idx, 'total_driving_duration_min'] = total_duration

    return results


# Calculate durations
event_markers_with_durations = calculate_driving_durations(event_markers_df)

# Display results
print("=== DRIVING PHASE DURATIONS ===")
duration_cols = ['file_name', 'total_duration_min', 'block1_duration_min',
                 'block2_duration_min', 'total_driving_duration_min']

available_cols = [
    col for col in duration_cols if col in event_markers_with_durations.columns]
print(event_markers_with_durations[available_cols].head(10))

# %%
# Summary statistics
print("\n=== DURATION STATISTICS ===")
duration_stats = ['block1_duration_min',
                  'block2_duration_min', 'total_driving_duration_min']
available_stats = [
    col for col in duration_stats if col in event_markers_with_durations.columns]

if available_stats:
    summary = event_markers_with_durations[available_stats].describe()
    print(summary.round(2))

Found 63 .acq files in ../.data/AdVitam/Exp4/Raw/Physio/BioPac/
Processing: 01_AC16.acq
Processing: 02_DU16.acq
Processing: 03_AC16.acq
Processing: 04_DC16.acq
Processing: 05_AC10.acq
Processing: 06_AU16.acq
Processing: 07_DU16.acq
Processing: 08_AC10.acq
Processing: 09_DC10.acq
Processing: 10_AU10.acq
Processing: 11_AU16.acq
Processing: 12_AC10.acq
Processing: 13_DU10.acq
Processing: 14_AC16.acq
Processing: 15_AU10.acq
Processing: 16_AU16.acq
Processing: 17_AU10.acq
Processing: 18_DU16.acq
Processing: 19_AC16.acq
Processing: 20_DC16.acq
Processing: 21_DC16.acq
Processing: 22_DC10.acq
Processing: 23_DU16.acq
Processing: 24_DU10.acq
Processing: 25_AC10.acq
Processing: 26_AU10.acq
Processing: 27_DU16.acq
Processing: 28_DC16.acq
Processing: 29_DU10.acq
Processing: 30_DC10.acq
Processing: 31_AC10.acq
Processing: 32_DU10.acq
Processing: 33_AU16.acq
Processing: 34_DU16.acq
Processing: 35_AU10.acq
Processing: 36_AC16.acq
Processing: 37_DC10.acq
Processing: 38_AU16.acq
Processing: 39_DC16.acq


In [ ]:

# %%
def load_biopac_signals(file_path):
    """
    Load BioPac .acq file and extract physiological signals
    """
    data = bioread.read_file(file_path)
    
    signals = {}
    sampling_rate = None
    
    for i, channel in enumerate(data.channels):
        if sampling_rate is None:
            sampling_rate = channel.samples_per_second
        
        # Map channels based on common naming patterns
        channel_name = channel.name.upper()
        if any(ecg_term in channel_name for ecg_term in ['ECG', 'EKG']):
            signals['ECG'] = channel.data
        elif any(eda_term in channel_name for eda_term in ['EDA', 'GSR', 'SC']):
            signals['EDA'] = channel.data
        elif any(resp_term in channel_name for resp_term in ['RESP', 'RSP', 'BREATHING']):
            signals['RESP'] = channel.data
    
    return signals, sampling_rate, data

In [ ]:
event_markers_with_durations

,file_name,file_path,total_duration_min,sampling_rate,n_channels,channels,Segment 1_time_sec,Segment 1_time_min,Baseline Start_time_sec,Baseline Start_time_min,...,Block3 Start_time_min,Block3 End_time_sec,Block3 End_time_min,ST End_time_sec,ST End_time_min,Segment 2_time_sec,Segment 2_time_min,total_driving_duration_min,block1_start_min,block2_end_min
0,01_AC16.acq,../.data/AdVitam/Exp4/Raw/Physio/BioPac/01_AC1...,78.775000,1000.0,3,"[EDA (0 - 35 Hz), ECG (.5 - 35 Hz), Respiration]",0.0,0.0,140.985,2.349750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.050000,18.015417,78.065417
1,02_DU16.acq,../.data/AdVitam/Exp4/Raw/Physio/BioPac/02_DU1...,71.751583,1000.0,3,"[EDA (0 - 35 Hz), ECG (.5 - 35 Hz), Respiration]",0.0,0.0,193.115,3.218583,...,38.647917,4042.175,67.369583,NaN,NaN,NaN,NaN,19.470583,16.478500,35.949083
2,03_AC16.acq,../.data/AdVitam/Exp4/Raw/Physio/BioPac/03_AC1...,81.401750,1000.0,3,"[EDA (0 - 35 Hz), ECG (.5 - 35 Hz), Respiration]",0.0,0.0,148.370,2.472833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.489083,16.368833,80.857917
3,04_DC16.acq,../.data/AdVitam/Exp4/Raw/Physio/BioPac/04_DC1...,80.326667,1000.0,3,"[EDA (0 - 35 Hz), ECG (.5 - 35 Hz), Respiration]",0.0,0.0,176.525,2.942083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.646167,18.613667,80.259833
4,05_AC10.acq,../.data/AdVitam/Exp4/Raw/Physio/BioPac/05_AC1...,65.391250,1000.0,3,"[EDA (0 - 35 Hz), ECG (.5 - 35 Hz), Respiration]",0.0,0.0,176.020,2.933667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.377917,16.545250,60.923167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,59_DC16.acq,../.data/AdVitam/Exp4/Raw/Physio/BioPac/59_DC1...,75.571333,1000.0,3,"[EDA (0 - 35 Hz), ECG (.5 - 35 Hz), Respiration]",0.0,0.0,93.885,1.564750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.341083,14.699083,75.040167
59,60_DU16.acq,../.data/AdVitam/Exp4/Raw/Physio/BioPac/60_DU1...,89.461500,1000.0,3,"[EDA (0 - 35 Hz), ECG (.5 - 35 Hz), Respiration]",0.0,0.0,74.170,1.236167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.412250,13.219000,88.631250
60,61_DC10.acq,../.data/AdVitam/Exp4/Raw/Physio/BioPac/61_DC1...,77.590083,1000.0,3,"[EDA (0 - 35 Hz), ECG (.5 - 35 Hz), Respiration]",0.0,0.0,179.750,2.995833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.908917,15.200667,77.109583
61,62_AU16.acq,../.data/AdVitam/Exp4/Raw/Physio/BioPac/62_AU1...,92.597167,1000.0,3,"[EDA (0 - 35 Hz), ECG (.5 - 35 Hz), Respiration]",0.0,0.0,91.080,1.518000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.556667,13.989750,92.546417


In [ ]:
# %%
# Save event marker analysis
event_markers_with_durations.to_csv('output/event_markers_analysis.csv', index=False)
print("Event marker analysis saved to 'event_markers_analysis.csv'")


print("\n=== ANALYSIS COMPLETE ===")
print("Check the saved CSV files for detailed results.")

Event marker analysis saved to 'event_markers_analysis.csv'

=== ANALYSIS COMPLETE ===
Check the saved CSV files for detailed results.


In [ ]:
def diagnose_hrv_missing_values_exp4_corrected(data_directory, target_features=['HRV_SDANN1_Dr', 'HRV_SDANN5_Dr']):
    """
    Diagnose missing HRV values in AdVitam Exp4 dataset with CORRECTED segmentation logic
    
    This function implements the correct segmentation approach as used in the original
    AdVitam Exp4 preprocessing pipeline, based on dataset documentation analysis.
    
    CORRECTED SEGMENTATION LOGIC:
    - Each BLOCK (Block1, Block2) is segmented separately
    - segm_1: Block1 → 1 segment, Block2 → 1 segment (total: 2 segments)
    - segm_5: Block1 → 5 segments, Block2 → 5 segments (total: 10 segments)
    
    HRV DURATION REQUIREMENTS (from NeuroKit2 documentation):
    - SDANN1: Requires minimum 3 minutes of signal
    - SDANN2: Requires minimum 6 minutes of signal  
    - SDANN5: Requires minimum 15 minutes of signal
    
    References:
    - NeuroKit2 HRV Documentation: https://neuropsychology.github.io/NeuroKit/functions/hrv.html
    - AdVitam Dataset Documentation: Exp4_README.md
    
    Parameters:
    -----------
    data_directory : str
        Path to directory containing Exp4 .acq files
    target_features : list
        List of HRV features to analyze for missing values
        
    Returns:
    --------
    pd.DataFrame
        Comprehensive results showing missing feature counts per file/segment
    """
    
    print("=== AdVitam Exp4 HRV Missing Values Analysis (CORRECTED) ===")
    print(f"Target features: {target_features}")
    print("✓ CORRECTED: Each block segmented separately")
    print(f"✓ Analyzing directory: {data_directory}")
    print()
    
    # HRV feature minimum duration requirements from NeuroKit2 documentation
    # Source: https://neuropsychology.github.io/NeuroKit/functions/hrv.html
    # "Note that these indices require a minimal duration of signal to be computed 
    # (3, 6 and 15 minutes respectively) and will be silently skipped if the data provided is too short"
    DURATION_REQUIREMENTS = {
        'SDANN1': 3,   # Needs 3-minute minimum signal duration
        'SDANN2': 6,   # Needs 6-minute minimum signal duration  
        'SDANN5': 15,  # Needs 15-minute minimum signal duration
    }
    
    # Expected experimental structure from Exp4_README.md
    EXPECTED_BLOCK_DURATION = 30  # Each block should be ~30 minutes
    
    data_path = Path(data_directory)
    acq_files = list(data_path.glob("*.acq"))
    
    if not acq_files:
        print(f"❌ ERROR: No .acq files found in {data_directory}")
        return None
    
    print(f"✓ Found {len(acq_files)} .acq files to analyze")
    print()
    
    # Results storage - comprehensive tracking
    all_segments_results = []
    file_summary = []
    
    # Process each file (limiting to first 3 for testing)
    for file_idx, file_path in enumerate(sorted(acq_files[:3])):
        print(f"{'='*80}")
        print(f"PROCESSING FILE {file_idx + 1}/{min(3, len(acq_files))}: {file_path.name}")
        print(f"{'='*80}")
        
        try:
            # STEP 1: Extract experimental phase markers from .acq file
            # This reads the event markers that define Block1 and Block2 boundaries
            file_data = extract_event_markers_from_file(file_path)
            
            # STEP 2: Locate Block1 and Block2 time boundaries
            # According to Exp4_README.md, each scenario lasts ~30 minutes
            block1_start = None
            block1_end = None
            block2_start = None  
            block2_end = None
            
            # Parse experimental markers (may have slight naming variations)
            for key, value in file_data.items():
                if 'Block1 Start_time_min' in key:
                    block1_start = value
                elif 'Block1 End_time_min' in key or 'Block1 End _time_min' in key:
                    block1_end = value
                elif 'Block2 Start_time_min' in key:
                    block2_start = value
                elif 'Block2 End_time_min' in key or 'Block2 End _time_min' in key:
                    block2_end = value
            
            # Validate that all block boundaries were found
            if None in [block1_start, block1_end, block2_start, block2_end]:
                print("❌ ERROR: Cannot locate all required block boundaries")
                print("Available time markers:")
                for key in file_data.keys():
                    if 'time_min' in key:
                        print(f"   {key}: {file_data[key]:.2f} min")
                continue
            
            # Verify block durations match expected values
            block1_duration = block1_end - block1_start
            block2_duration = block2_end - block2_start
            
            print(f"📊 EXPERIMENTAL BLOCK TIMINGS:")
            print(f"   Block1: {block1_start:.2f} → {block1_end:.2f} min (duration: {block1_duration:.2f} min)")
            print(f"   Block2: {block2_start:.2f} → {block2_end:.2f} min (duration: {block2_duration:.2f} min)")
            
            # Warn if durations deviate significantly from expected 30 minutes
            for block_name, duration in [("Block1", block1_duration), ("Block2", block2_duration)]:
                if abs(duration - EXPECTED_BLOCK_DURATION) > 5:  # 5-minute tolerance
                    print(f"⚠️  WARNING: {block_name} duration ({duration:.1f} min) deviates from expected {EXPECTED_BLOCK_DURATION} min")
            
            # STEP 3: Load physiological signals from BioPac file
            signals, sampling_rate, raw_data = load_biopac_signals(file_path)
            
            if 'ECG' not in signals:
                print("❌ ERROR: ECG signal not found in file")
                continue
            
            print(f"✓ Loaded ECG signal (sampling rate: {sampling_rate} Hz)")
            
            # STEP 4: Define experimental blocks for processing
            blocks = [
                ("Block1", block1_start, block1_end),
                ("Block2", block2_start, block2_end)
            ]
            
            # STEP 5: Test different segmentation levels
            # This matches the segmentation approach used in the original preprocessing
            segmentation_configs = [
                ("segm_1", 1),   # Each block as single segment
                ("segm_5", 5)    # Each block divided into 5 segments
            ]
            
            file_missing_summary = {
                'file': file_path.name,
                'block1_duration': block1_duration,
                'block2_duration': block2_duration
            }
            
            for segm_name, n_segments in segmentation_configs:
                print(f"\n🔍 {segm_name.upper()} SEGMENTATION ANALYSIS:")
                print(f"   Strategy: Each block divided into {n_segments} segment(s)")
                
                segm_results = []
                
                # Process each experimental block separately (CORRECTED APPROACH)
                for block_name, block_start, block_end in blocks:
                    print(f"\n   --- {block_name} Processing ---")
                    
                    # Extract ECG signal for this specific block
                    start_sample = int(block_start * 60 * sampling_rate)
                    end_sample = int(block_end * 60 * sampling_rate)
                    
                    # Handle potential signal length issues
                    if end_sample > len(signals['ECG']):
                        print(f"   ⚠️  Adjusting end sample from {end_sample} to {len(signals['ECG'])}")
                        end_sample = len(signals['ECG'])
                    
                    block_signal = signals['ECG'][start_sample:end_sample]
                    block_duration = len(block_signal) / (sampling_rate * 60)
                    
                    print(f"   Block extracted: {block_duration:.2f} minutes of ECG data")
                    
                    # STEP 6: Apply segmentation to this block
                    # CORRECTED: Segmentation applied per block, not across entire experiment
                    if n_segments == 1:
                        # Single segment = entire block
                        segments = [block_signal]
                        segment_labels = [f"{block_name}_whole"]
                    else:
                        # Multiple segments: divide block into equal parts
                        segment_length = len(block_signal) // n_segments
                        segments = []
                        segment_labels = []
                        
                        for i in range(n_segments):
                            start_idx = i * segment_length
                            end_idx = start_idx + segment_length
                            
                            # Last segment includes any remainder samples
                            if i == n_segments - 1:
                                end_idx = len(block_signal)
                            
                            segments.append(block_signal[start_idx:end_idx])
                            segment_labels.append(f"{block_name}_seg{i + 1}")
                    
                    # STEP 7: Analyze each segment for HRV calculation feasibility
                    for seg_idx, (segment, segment_label) in enumerate(zip(segments, segment_labels)):
                        segment_duration = len(segment) / (sampling_rate * 60)
                        
                        print(f"\n     {segment_label}: {segment_duration:.2f} minutes")
                        
                        # Initialize segment result tracking
                        segment_result = {
                            'file': file_path.name,
                            'segmentation': segm_name,
                            'block': block_name,
                            'segment_id': seg_idx + 1,
                            'segment_label': segment_label,
                            'duration_min': segment_duration,
                            'total_samples': len(segment),
                            'r_peaks_detected': 0,
                            'hrv_calculation_attempted': False,
                            'hrv_calculation_successful': False,
                            'missing_features_count': 0,
                            'missing_features_list': []
                        }
                        
                        # Add individual feature tracking columns
                        for feature in target_features:
                            segment_result[f'{feature}_calculable'] = False
                            segment_result[f'{feature}_value'] = None
                            segment_result[f'{feature}_missing_reason'] = None
                        
                        # STEP 8: Predict which features will be missing based on duration
                        predicted_missing = []
                        for feature in target_features:
                            # Extract base feature name (remove _Dr suffix)
                            base_feature = feature.replace('_Dr', '').replace('HRV_', '')
                            
                            if base_feature in DURATION_REQUIREMENTS:
                                required_duration = DURATION_REQUIREMENTS[base_feature]
                                if segment_duration < required_duration:
                                    predicted_missing.append(feature)
                                    segment_result[f'{feature}_missing_reason'] = f"Duration {segment_duration:.1f}min < required {required_duration}min"
                        
                        if predicted_missing:
                            print(f"     📉 PREDICTED MISSING: {predicted_missing}")
                            print(f"        Reason: Insufficient duration for NeuroKit2 calculation")
                        
                        # STEP 9: Attempt actual HRV calculation
                        try:
                            segment_result['hrv_calculation_attempted'] = True
                            
                            # ECG preprocessing using NeuroKit2
                            ecg_cleaned = nk.ecg_clean(segment, sampling_rate=sampling_rate)
                            _, rpeaks_info = nk.ecg_peaks(ecg_cleaned, sampling_rate=sampling_rate)
                            rpeaks = rpeaks_info['ECG_R_Peaks']
                            
                            segment_result['r_peaks_detected'] = len(rpeaks)
                            print(f"     💓 R-peaks detected: {len(rpeaks)}")
                            
                            # Require minimum number of R-peaks for reliable HRV calculation
                            if len(rpeaks) < 10:
                                print(f"     ❌ Insufficient R-peaks (minimum 10 required)")
                                segment_result['hrv_calculation_successful'] = False
                                for feature in target_features:
                                    segment_result[f'{feature}_missing_reason'] = f"Insufficient R-peaks ({len(rpeaks)} < 10)"
                            else:
                                # Calculate HRV features using NeuroKit2
                                hrv_features = nk.hrv_time(rpeaks, sampling_rate=sampling_rate, show=False)
                                segment_result['hrv_calculation_successful'] = True
                                
                                print(f"     ✅ HRV calculation successful")
                                
                                # Check each target feature
                                missing_count = 0
                                missing_list = []
                                
                                for feature in target_features:
                                    # Map feature name to NeuroKit2 output column
                                    base_feature = feature.replace('_Dr', '').replace('HRV_', '')
                                    nk_column = f'HRV_{base_feature}'
                                    
                                    if nk_column in hrv_features.columns:
                                        value = hrv_features[nk_column].iloc[0]
                                        segment_result[f'{feature}_value'] = value
                                        
                                        if pd.isna(value):
                                            print(f"       ❌ {feature}: NaN (duration insufficient)")
                                            segment_result[f'{feature}_calculable'] = False
                                            segment_result[f'{feature}_missing_reason'] = "NeuroKit2 returned NaN - duration insufficient"
                                            missing_count += 1
                                            missing_list.append(feature)
                                        else:
                                            print(f"       ✅ {feature}: {value:.4f}")
                                            segment_result[f'{feature}_calculable'] = True
                                    else:
                                        print(f"       ❌ {feature}: Column not found in NeuroKit2 output")
                                        segment_result[f'{feature}_calculable'] = False
                                        segment_result[f'{feature}_missing_reason'] = "Feature not in NeuroKit2 output"
                                        missing_count += 1
                                        missing_list.append(feature)
                                
                                segment_result['missing_features_count'] = missing_count
                                segment_result['missing_features_list'] = missing_list
                        
                        except Exception as e:
                            print(f"     ❌ HRV calculation failed: {str(e)}")
                            segment_result['hrv_calculation_successful'] = False
                            
                            # Mark all features as missing due to calculation error
                            for feature in target_features:
                                segment_result[f'{feature}_calculable'] = False
                                segment_result[f'{feature}_missing_reason'] = f"Calculation error: {str(e)}"
                            
                            segment_result['missing_features_count'] = len(target_features)
                            segment_result['missing_features_list'] = target_features.copy()
                        
                        all_segments_results.append(segment_result)
                        segm_results.append(segment_result)
                
                # Summarize results for this segmentation level
                total_segments = len(segm_results)
                successful_calculations = sum(1 for r in segm_results if r['hrv_calculation_successful'])
                total_missing_features = sum(r['missing_features_count'] for r in segm_results)
                
                print(f"\n   📊 {segm_name} Summary:")
                print(f"      Total segments: {total_segments}")
                print(f"      Successful HRV calculations: {successful_calculations}/{total_segments}")
                print(f"      Total missing features: {total_missing_features}")
                
                # Add to file summary
                file_missing_summary[f'{segm_name}_total_segments'] = total_segments
                file_missing_summary[f'{segm_name}_successful_calculations'] = successful_calculations
                file_missing_summary[f'{segm_name}_total_missing_features'] = total_missing_features
            
            file_summary.append(file_missing_summary)
            
        except Exception as e:
            print(f"❌ CRITICAL ERROR processing {file_path.name}: {str(e)}")
            continue
    
    # STEP 10: Generate comprehensive analysis report
    print(f"\n{'='*80}")
    print(f"COMPREHENSIVE ANALYSIS RESULTS")
    print(f"{'='*80}")
    
    if not all_segments_results:
        print("❌ No results to analyze - check file paths and data integrity")
        return None
    
    # Convert to DataFrame for analysis
    results_df = pd.DataFrame(all_segments_results)
    
    print(f"✅ Analysis completed successfully")
    print(f"   Total segments analyzed: {len(results_df)}")
    print(f"   Files processed: {results_df['file'].nunique()}")
    print(f"   Segmentation levels tested: {results_df['segmentation'].nunique()}")
    
    # STEP 11: Segmentation comparison analysis
    print(f"\n📊 SEGMENTATION LEVEL COMPARISON:")
    
    for segm in results_df['segmentation'].unique():
        segm_data = results_df[results_df['segmentation'] == segm]
        
        print(f"\n{segm.upper()}:")
        print(f"  Total segments: {len(segm_data)}")
        print(f"  Successful HRV calculations: {segm_data['hrv_calculation_successful'].sum()}/{len(segm_data)}")
        
        for feature in target_features:
            calculable_col = f'{feature}_calculable'
            if calculable_col in segm_data.columns:
                calculable_count = segm_data[calculable_col].sum()
                total_count = len(segm_data)
                success_rate = (calculable_count / total_count * 100) if total_count > 0 else 0
                missing_rate = 100 - success_rate
                
                print(f"  {feature}: {calculable_count}/{total_count} calculable ({success_rate:.1f}% success, {missing_rate:.1f}% missing)")
    
    # STEP 12: Missing features summary table  
    print(f"\n📋 MISSING FEATURES SUMMARY TABLE:")
    print(f"{'Feature':<20} {'SEGM_1 Missing %':<15} {'SEGM_5 Missing %':<15} {'Difference':<12} {'Explanation'}")
    print("-" * 100)
    
    for feature in target_features:
        calculable_col = f'{feature}_calculable'
        
        # Calculate missing rates by segmentation level
        segm1_data = results_df[results_df['segmentation'] == 'segm_1']
        segm5_data = results_df[results_df['segmentation'] == 'segm_5']
        
        segm1_missing_rate = 0
        segm5_missing_rate = 0
        
        if len(segm1_data) > 0 and calculable_col in segm1_data.columns:
            segm1_missing_rate = ((len(segm1_data) - segm1_data[calculable_col].sum()) / len(segm1_data) * 100)
        
        if len(segm5_data) > 0 and calculable_col in segm5_data.columns:
            segm5_missing_rate = ((len(segm5_data) - segm5_data[calculable_col].sum()) / len(segm5_data) * 100)
        
        difference = segm5_missing_rate - segm1_missing_rate
        
        # Provide explanation based on duration requirements
        base_feature = feature.replace('_Dr', '').replace('HRV_', '')
        if base_feature in DURATION_REQUIREMENTS:
            req_duration = DURATION_REQUIREMENTS[base_feature]
            if req_duration <= 6:  # Should work with segm_5 (6-min segments)
                explanation = "Duration sufficient"
            else:  # SDANN5 case (needs 15 min)
                explanation = f"Needs {req_duration}min (segm_5 only ~6min)"
        else:
            explanation = "Unknown requirement"
        
        print(f"{feature:<20} {segm1_missing_rate:<15.1f} {segm5_missing_rate:<15.1f} {difference:<12.1f} {explanation}")
    
    # STEP 13: Duration analysis
    print(f"\n📏 SEGMENT DURATION ANALYSIS:")
    duration_summary = results_df.groupby('segmentation')['duration_min'].agg(['mean', 'min', 'max', 'std']).round(2)
    print(duration_summary)
    
    # STEP 14: File-level summary
    print(f"\n📁 FILE-LEVEL SUMMARY:")
    file_summary_df = pd.DataFrame(file_summary)
    if not file_summary_df.empty:
        print(file_summary_df.to_string(index=False))
    
    print(f"\n✅ Analysis complete. Results DataFrame contains {len(results_df)} rows with detailed missing feature information.")
    
    return results_df

# Example usage
if __name__ == "__main__":
    # Run the corrected diagnostic analysis
    results_df = diagnose_hrv_missing_values_exp4_corrected(
        data_directory="../.data/AdVitam/Exp4/Raw/Physio/BioPac/",
        target_features=['HRV_SDANN1_Dr', 'HRV_SDANN5_Dr']
    )
    
    if results_df is not None:
        print(f"\nDataFrame shape: {results_df.shape}")
        print(f"Columns: {list(results_df.columns)}")
        
        # Show missing features count per file/segment
        missing_summary = results_df.groupby(['file', 'segmentation'])['missing_features_count'].agg(['sum', 'mean']).round(2)
        print(f"\nMissing features per file/segmentation:\n{missing_summary}")

=== AdVitam Exp4 HRV Missing Values Analysis (CORRECTED) ===
Target features: ['HRV_SDANN1_Dr', 'HRV_SDANN5_Dr']
✓ CORRECTED: Each block segmented separately
✓ Analyzing directory: ../.data/AdVitam/Exp4/Raw/Physio/BioPac/

✓ Found 63 .acq files to analyze

PROCESSING FILE 1/3: 23_DU16.acq
📊 EXPERIMENTAL BLOCK TIMINGS:
   Block1: 13.90 → 38.54 min (duration: 24.64 min)
   Block2: 42.95 → 72.73 min (duration: 29.78 min)
⚠️  WARNING: Block1 duration (24.6 min) deviates from expected 30 min
✓ Loaded ECG signal (sampling rate: 1000.0 Hz)

🔍 SEGM_1 SEGMENTATION ANALYSIS:
   Strategy: Each block divided into 1 segment(s)

   --- Block1 Processing ---
   Block extracted: 24.64 minutes of ECG data

     Block1_whole: 24.64 minutes
     💓 R-peaks detected: 1755
     ✅ HRV calculation successful
       ✅ HRV_SDANN1_Dr: 31.0630
       ✅ HRV_SDANN5_Dr: 22.2639

   --- Block2 Processing ---
   Block extracted: 29.78 minutes of ECG data

     Block2_whole: 29.78 minutes
     💓 R-peaks detected: 2180
